In [ ]:
from spottunet.dataset.cc359 import *
from spottunet.split import one2all
from spottunet.torch.module.unet import UNet2D
from spottunet.utils import sdice
from dpipe.im.metrics import dice_score

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.tensorboard import SummaryWriter
from torch.cuda.amp import autocast
from torch.cuda.amp import GradScaler 

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

from monai import transforms as T
from monai.transforms import Compose, apply_transform
from fastprogress.fastprogress import master_bar, progress_bar

import json
import nibabel as nib
import pandas as pd
import numpy as np
from scipy import ndimage
from dpipe.im.shape_ops import zoom
import cv2
import os
import gc
from collections import defaultdict
from PIL import Image
import seaborn as sns

from pathlib import Path
from dpipe.io import load

import joypy
from matplotlib import cm

%reload_ext autoreload
%autoreload 2
from configs.config import CFG
from dataset.dataloader import *
from dataset.dataloader_utils import *

import matplotlib.pyplot as plt

cc359_df = pd.read_csv(f"{CFG.dataset_path}/meta.csv",delimiter=",")
data_path="/home/mlk/cc359"

def id_to_scanner(id):
    df = cc359_df[cc359_df['id']==id]
    return df['tomograph_model'].values[0] + str(df['tesla_value'].values[0])
test = "CC0231"

id_to_scanner(test)
cc359_df.head()

In [ ]:
from sklearn.mixture import GaussianMixture

fold = 0

df_fold = cc359_df[cc359_df['fold']==fold]

x_brain_list = []
matter_list = []
gm_list = []
wm_list = []
csf_list = []
x_ids = []
for mri,mask,ids in zip(df_fold['MRI_scaled_voxel_spacing'],df_fold['brain_mask'], df_fold['id']):
    gm_fn = mri.replace("images_scaled_voxel_spacing","fcm_norm_gm").replace(".nii.gz","gm_membership.nii.gz")
    wm_fn = mri.replace("images_scaled_voxel_spacing","fcm_norm_wm").replace(".nii.gz","wm_membership.nii.gz")
    csf_fn = mri.replace("images_scaled_voxel_spacing","fcm_norm_csf").replace(".nii.gz","csf_membership.nii.gz")
    robex = mri.replace("images_scaled_voxel_spacing","robex_masks_svsp")
    x = nib.load(f'{data_path}/{mri}').get_fdata()
    x = scale_mri(x, percentile=None)
    y = nib.load(f'{data_path}/{mask}').get_fdata() > 0.
    
    gm_mask = nib.load(f'{data_path}/{gm_fn}').get_fdata() > 0.5
    wm_mask = nib.load(f'{data_path}/{wm_fn}').get_fdata() > 0.5
    csf_mask = nib.load(f'{data_path}/{csf_fn}').get_fdata() > 0.3
    
    scan = nib.load(f'{data_path}/{mri}')
    mask = nib.load(f'{data_path}/{robex}').get_fdata() > 0.5
    
    lbl = GaussianMixture(n_components=3, random_state=0).fit_predict(x[mask][:,None])
    print(lbl.shape, lbl)
    plt.imshow(wm_mask[140])
    x_brain = x[y]
    x_brain_list.append(x_brain)
    matter_list.append([x[gm_mask],x[wm_mask],x[csf_mask]])
    gm_list.append(x[gm_mask])
    wm_list.append(x[wm_mask])
    csf_list.append(x[csf_mask])
    x_ids.append(ids)
    break

In [ ]:
sns.histplot([x[mask][lbl==0],x[mask][lbl==1],x[mask][lbl==2]], kde=True)

In [ ]:
d = dict(CC0001=
             dict(gm=list(gm_list[0]),wm=list(wm_list[0]),csf=csf_list[0]),
         CC0002=
             dict(gm=list(gm_list[1]),wm=list(wm_list[1]),csf=csf_list[1])
        )

fig, axes = joypy.joyplot(d, range_style='own', 
                          grid="y", linewidth=1, legend=False, figsize=(4,5),
                          title="", x_range=[-0.1,0.8])

In [ ]:
from pathlib import Path
from dpipe.io import load
path_base = Path('baseline_results/baseline_focal_lovasz_SGD_None_None')

meta = pd.read_csv(f"meta.csv",delimiter=",", index_col='id')
meta.head()

all_sdices = []
for s in sorted(cc359_df['fold'].unique()):
    sdices = load(path_base / f'mode_{s}/sdice_score.json')
    all_sdices.append(sdices)

In [ ]:
fold = 0

df_fold = cc359_df[cc359_df['fold']==fold]

x_brain_list = []
x_ids = []
for mri,mask,ids in zip(df_fold['MRI'],df_fold['brain_mask'], df_fold['id']):
    x = nib.load(f'{data_path}/{mri}').get_fdata()
    x = scale_mri(x, percentile=None)
    y = nib.load(f'{data_path}/{mask}').get_fdata() > 0.
    
    x_brain = x[y]
    x_brain_list.append(x_brain)
    x_ids.append(ids)
fig, axes = joypy.joyplot(x_brain_list, labels=x_ids, range_style='own', 
                          grid="y", linewidth=1, legend=False, figsize=(4,10),
                          title="", x_range=[-0.1,0.7],
                          colormap=cm.Blues_r)

In [ ]:
fold = 1

df_fold = cc359_df[cc359_df['fold']==fold]

x_brain_list = []
x_ids = []
for mri,mask,ids in zip(df_fold['MRI'],df_fold['brain_mask'], df_fold['id']):
    x = nib.load(f'{data_path}/{mri}').get_fdata()
    x = scale_mri(x, percentile=None)
    y = nib.load(f'{data_path}/{mask}').get_fdata() > 0.
    
    x_brain = x[y]
    x_brain_list.append(x_brain)
    x_ids.append(ids)
fig, axes = joypy.joyplot(x_brain_list, labels=x_ids, range_style='own', 
                          grid="y", linewidth=1, legend=False, figsize=(4,10),
                          title="", x_range=[-0.1,0.8],
                          colormap=cm.Blues_r)

In [ ]:
fold = 2

df_fold = cc359_df[cc359_df['fold']==fold]

x_brain_list = []
x_ids = []
for mri,mask,ids in zip(df_fold['MRI'],df_fold['brain_mask'], df_fold['id']):
    x = nib.load(f'{data_path}/{mri}').get_fdata()
    x = scale_mri(x, percentile=None)
    y = nib.load(f'{data_path}/{mask}').get_fdata() > 0.
    
    x_brain = x[y]
    x_brain_list.append(x_brain)
    x_ids.append(ids)
fig, axes = joypy.joyplot(x_brain_list, labels=x_ids, range_style='own', 
                          grid="y", linewidth=1, legend=False, figsize=(4,10),
                          title="", x_range=[-0.1,0.8],
                          colormap=cm.Blues_r)

In [ ]:
fold = 3

df_fold = cc359_df[cc359_df['fold']==fold]

x_brain_list = []
x_ids = []
for mri,mask,ids in zip(df_fold['MRI'],df_fold['brain_mask'], df_fold['id']):
    x = nib.load(f'{data_path}/{mri}').get_fdata()
    x = scale_mri(x, percentile=None)
    y = nib.load(f'{data_path}/{mask}').get_fdata() > 0.
    
    x_brain = x[y]
    x_brain_list.append(x_brain)
    x_ids.append(ids)
fig, axes = joypy.joyplot(x_brain_list, labels=x_ids, range_style='own', 
                          grid="y", linewidth=1, legend=False, figsize=(4,10),
                          title="", x_range=[-0.1,0.8],
                          colormap=cm.Blues_r)

In [ ]:
fold = 4

df_fold = cc359_df[cc359_df['fold']==fold]

x_brain_list = []
x_ids = []
for mri,mask,ids in zip(df_fold['MRI'],df_fold['brain_mask'], df_fold['id']):
    x = nib.load(f'{data_path}/{mri}').get_fdata()
    x = scale_mri(x, percentile=None)
    y = nib.load(f'{data_path}/{mask}').get_fdata() > 0.
    
    x_brain = x[y]
    x_brain_list.append(x_brain)
    x_ids.append(ids)
fig, axes = joypy.joyplot(x_brain_list, labels=x_ids, range_style='own', 
                          grid="y", linewidth=1, legend=False, figsize=(4,10),
                          title="", x_range=[-0.1,0.8],
                          colormap=cm.Blues_r)

In [ ]:
fold = 5

df_fold = cc359_df[cc359_df['fold']==fold]

x_brain_list = []
x_ids = []
for mri,mask,ids in zip(df_fold['MRI'],df_fold['brain_mask'], df_fold['id']):
    x = nib.load(f'{data_path}/{mri}').get_fdata()
    x = scale_mri(x, percentile=None)
    y = nib.load(f'{data_path}/{mask}').get_fdata() > 0.
    
    x_brain = x[y]
    x_brain_list.append(x_brain)
    x_ids.append(ids)
fig, axes = joypy.joyplot(x_brain_list, labels=x_ids, range_style='own', 
                          grid="y", linewidth=1, legend=False, figsize=(4,10),
                          title="", x_range=[-0.1,0.8],
                          colormap=cm.Blues_r)

In [ ]:
fold = 5

df_fold = cc359_df[cc359_df['fold']==fold]

x_brain_list = []
x_ids = []
for mri,mask,ids in zip(df_fold['MRI'],df_fold['brain_mask'], df_fold['id']):
    x = nib.load(f'{data_path}/{mri}').get_fdata()
    x = scale_mri(x, percentile=None)
    y = nib.load(f'{data_path}/{mask}').get_fdata() > 0.
    
    x_brain = x[y]
    x_brain_list.append(x_brain)
    x_ids.append(ids)
fig, axes = joypy.joyplot(x_brain_list, labels=x_ids, range_style='own', 
                          grid="y", linewidth=1, legend=False, figsize=(4,10),
                          title="", x_range=[-0.1,0.8],
                          colormap=cm.Blues_r)

In [ ]:
def get_data_ids(fold):
    df_fold = cc359_df[cc359_df['fold']==fold]
    x_brain_list = []
    x_ids = []
    for mri,mask,ids in zip(df_fold['MRI'],df_fold['brain_mask'], df_fold['id']):
        x = nib.load(f'{data_path}/{mri}').get_fdata()
        x = scale_mri(x, percentile=None)
        y = nib.load(f'{data_path}/{mask}').get_fdata() > 0.

        x_brain = x[y]
        x_brain_list.append(x_brain)
        x_ids.append(ids)
    return x_brain_list,x_ids

In [ ]:
x,ids = get_data_ids(0)

In [ ]:
sns.histplot(np.concatenate(x)[::2])

In [ ]:
x_0,ids_0 = get_data_ids(0)
x_1,ids_1 = get_data_ids(1)
x_2,ids_2 = get_data_ids(2)
x_3,ids_3 = get_data_ids(3)
x_4,ids_4 = get_data_ids(4)
x_5,ids_5 = get_data_ids(5)

x_0 = np.concatenate(x_0)
x_1 = np.concatenate(x_1)
x_2 = np.concatenate(x_2)
x_3 = np.concatenate(x_3)
x_4 = np.concatenate(x_4)
x_5 = np.concatenate(x_5)

In [ ]:
sns.set_style("darkgrid")
sns.kdeplot(x_0[::2], label="Siemens1.5")
sns.kdeplot(x_1[::2], label="Siemens3")
sns.kdeplot(x_2[::2], label="Ge15")
sns.kdeplot(x_3[::2], label="Ge3")
sns.kdeplot(x_4[::2], label="Philips1.5")
sns.kdeplot(x_5[::2], label="Philips3")
plt.legend()
plt.savefig("kdes.png",dpi=300)
plt.show()